In [2]:
import requests
import time
import execjs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from multiprocessing import Process

In [4]:
def getAllCode():
    url = 'http://fund.eastmoney.com/js/fundcode_search.js'
    content = requests.get(url)
    jsContent = execjs.compile(content.text)
    rawData = jsContent.eval('r')
    allCode = []
    for code in rawData:
        allCode.append(code[0])
    return allCode


In [44]:
# fund_narrow_pd=pd.read_csv('./narrow/fund_narrow_pd.csv',low_memory=False, header = None)
# worth_pd文件 最长是 202001
# rateInSimilar_pd最长是040001
# grandTotal_pd 最长是 005503
grandTotal_pd=pd.read_csv('./narrow/grandTotal_pd.csv',low_memory=False, header = None)
rateInSimilar_pd=pd.read_csv('./narrow/rateInSimilar_pd.csv',low_memory=False, header = None)
worth_pd = pd.read_csv(open('./1-8926/san/worth_pd.csv','r'),low_memory=False, header = None)

In [66]:
# df.iloc = df['col2'].astype('float64')
worth_pd = worth_pd.fillna(0)
grandTotal_pd = grandTotal_pd.fillna(0)
rateInSimilar_pd = rateInSimilar_pd.fillna(0)

print(grandTotal_pd.info())
print(rateInSimilar_pd.info())
print(worth_pd.info())
# 拆分：每份基金份额分拆
# 拆分：每份基金份额折算
# 分红：每份派现金
# worth_pd.loc[:,2:].astype('float')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29316 entries, 0 to 29315
Columns: 136 entries, 0 to 135
dtypes: float64(134), int64(1), object(1)
memory usage: 30.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30452 entries, 0 to 30451
Columns: 4303 entries, 0 to 4302
dtypes: float64(4301), int64(1), object(1)
memory usage: 999.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38075 entries, 0 to 38074
Columns: 4345 entries, 0 to 4344
dtypes: float64(4343), int64(1), object(1)
memory usage: 1.2+ GB
None


In [70]:
# 找 N 年以上的基金
def get_year_funds(years, allCode):
    funds_year = []
#     allCode = ['000008']
    for code in allCode:
#         print(code)
        fund_day = rateInSimilar_pd.loc[(rateInSimilar_pd[0] == "时间戳") & (rateInSimilar_pd[1] ==  int(code))]
        if fund_day.shape[0] > 0:
            fund_day = fund_day.T
#             print(fund_day.shape)
#             print(fund_day)
            day_c = fund_day[fund_day.iloc[:,0] != 0].shape[0]
            if day_c >= 365 * years + 10:
#                 print("code",code,"day_c",day_c)
                funds_year.append(code)
    return funds_year

In [71]:
allCode = getAllCode()
funds_5y = get_year_funds(5,allCode)
pd.DataFrame(funds_5y).to_csv("./funds_5y.csv", header=False, index=False)

In [72]:
funds_3y = get_year_funds(3,allCode)
pd.DataFrame(funds_3y).to_csv("./funds_3y.csv", header=False, index=False)

In [74]:
print(len(funds_3y))

1705


In [75]:
# 近 N 年 同类绩效均值 n(0-1)
def get_similar_mean(year):
    if year <= 3:
        file_year = 3
    if year >3 and year <= 5:
        file_year = 5
    fund_similar_mean_list = []
    fund_similar_code_list = []
    fund_year_pd = pd.read_csv(
        './funds_' + str(file_year) + 'y.csv', low_memory=False, header=None)
    fund_year_list = fund_year_pd.values.tolist()
#     fund_year_list = [1,11]
    for code in fund_year_list:
        code = code[0]
        fund_similar = fund_narrow_pd.loc[(fund_narrow_pd[0] == "同类排名百分比")
                                          & (fund_narrow_pd[1] == code)]
        fund_similar.drop([0, 1, 2], axis=1, inplace=True) # 删除前三列无用数据
        fund_similar = fund_similar.T
        fund_similar = fund_similar[fund_similar.iloc[:, 0] > 0] # 只要有效数据
        if fund_similar.shape[0] >= year * 365:
            fund_similar = fund_similar.iloc[0: year*365 , :]  # 找 x 天内的数据,取前 n 行
#             print(fund_similar.shape)
            mean = fund_similar.mean(0).iloc[0] # 求平均
            fund_similar_code_list.append(code)
            fund_similar_mean_list.append(mean)
            fund_similar_mean_dict = {
                "code" : fund_similar_code_list,
               "同类平均值" : fund_similar_mean_list
            } 
    fund_similar_mean_pd =  pd.DataFrame(fund_similar_mean_dict)
    
    return fund_similar_mean_pd

In [76]:
def get_similar_top(similar_mean,top):
#     排序
    similar_mean.sort_values(by = '同类平均值', inplace=True, ascending=False)
#     获取长度
    pd_len = similar_mean.shape[0]
#     取前三分之一
    similar_top = similar_mean.head(round(pd_len * top))
    return similar_top

In [77]:
# 统计5 年来前 30%的基金
for i in range(1,6,1):
    fund_similar_mean_pd = get_similar_mean(i)
    fund_similar_top_pd = get_similar_top(fund_similar_mean_pd, 0.3)
    fund_similar_top_pd.to_csv("./fund_similar_top" + str(i) +"y_pd.csv", index=False)
# print(fund_similar_mean_pd)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


IndexError: single positional indexer is out-of-bounds

In [382]:
fund_similar_top_1y_pd = pd.read_csv('./fund_similar_top_1y_pd.csv',low_memory=False, header=None)
fund_similar_top_2y_pd = pd.read_csv('./fund_similar_top_2y_pd.csv',low_memory=False, header=None)
print(fund_similar_top_1y_pd)
print(fund_similar_top_2y_pd)
m = pd.merge(fund_similar_top_1y_pd,fund_similar_top_2y_pd,how='inner',on = 0)
# print(m)

          0          1
0    471060  81.245963
1    150106  76.939799
2    150123  75.330694
3       345  74.458584
4    110053  73.228776
5    450009  72.315973
6       577  70.838557
7    110022  70.710667
8    160222  70.462658
9    510630  70.428429
10   380009  70.293023
11   150117  70.136164
12   159928  69.852329
13   163210  69.741379
14   150199  69.674201
15   217022  69.633982
16   150190  69.574932
17   540006  69.560995
18      751  69.292110
19   150130  69.226438
20   100072  69.093443
21   150198  69.062731
22   160127  68.896018
23   519736  68.874721
24   150164  68.844539
25   512600  68.774804
26   180012  68.120274
27   166902  68.099909
28   519714  67.996365
29      527  67.969087
..      ...        ...
483   40040  54.725763
484     192  54.710384
485  165309  54.705068
486  165807  54.696721
487  740101  54.665927
488     187  54.648000
489  660004  54.646082
490  161606  54.640466
491  166011  54.627406
492  257030  54.620228
493  519661  54.618447
494      65

In [ ]:
# 测试代码分割线

In [338]:
# mean = fund_similar_mean_pd.iloc[0,:].mean(0)
# print(fund_similar_mean_pd)
fund_similar_mean_pd_t = fund_similar_mean_pd
fund_similar_mean_pd_t.sort_values(by = '同类平均值', inplace=True, ascending=False)
pd_len = fund_similar_mean_pd_t.shape[0]
print(pd_len * 0.3)

print(fund_similar_mean_pd_t.shape)
# fund_similar_mean_pd_t = fund_similar_mean_pd_t.iloc[0:round(pd_len * 0.3)]
fund_similar_mean_pd_t = fund_similar_mean_pd_t.head(round(pd_len * 0.3))
print(fund_similar_mean_pd_t.shape)
# round(pd_len * 0.3)
# .head(3)


251.7
(839, 2)
(252, 2)


In [323]:
fund_similar = fund_narrow_pd.loc[(fund_narrow_pd[0] == "同类排名百分比")
                                  & (fund_narrow_pd[1] == 100022)]
fund_similar.drop([0, 1, 2], axis=1, inplace=True)  # 删除前三列无用数据
fund_similar = fund_similar.T
fund_similar = fund_similar[fund_similar.iloc[:, 0] > 0]  # 只要有效数据
#         还需要找 x 天内的数据
print(fund_similar.shape)
# print(fund_similar.iloc[:,5*365].shape)
# fund_similar = fund_similar.iloc[5*365,:]
print(fund_similar.iloc[0:5*365,:].shape)
mean = fund_similar.mean(0).iloc[0]  # 求平均
print(mean.shape)
print(mean)
print(type(mean))

(3447, 1)
(1825, 1)
()
54.835787641427295
<class 'numpy.float64'>


In [239]:

fund_day = fund_narrow_pd.loc[(fund_narrow_pd[0] == "时间戳") & (fund_narrow_pd[1] ==  int("040023"))]
fund_day = fund_day.T
print("fund_day", fund_day.shape)

x1 = fund_day[fund_day.iloc[:,0] != 0]
print(x1.shape)
# print(x1)




fund_day (4344, 1)
(2002, 1)


Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]
Index: []

[0 rows x 4344 columns]
